In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

from tensorboardX import SummaryWriter


In [2]:
writer = SummaryWriter('runs/profiler')

In [6]:
df = pd.read_csv("/home/melinda/Examensarbete/DataSet/ThesisDataSet/Eclipse.csv")

In [ ]:
df = df[['sd','cl', 'pd', 'co', 'rp', 'os', 'bs', 'rs', 'pr', 'bsr']]
df.drop_duplicates(inplace=True)
print(df)

                                                     sd                  cl  \
0               PDE quickfix creates invalid @Since tag     Eclipse Project   
1     Grant access to projects storage service to th...  Eclipse Foundation   
2                  Add relation information to REST-API          Automotive   
3     Provide platform independent plug-in to set th...     Eclipse Project   
4     Inline method refacting reports "Inaccurate re...     Eclipse Project   
...                                                 ...                 ...   
8473                            JIPP for tools.shellwax  Eclipse Foundation   
8474  [13] Highlighting is broken when Unicode equiv...     Eclipse Project   
8475            duplicate configure fetch/push commands          Technology   
8476             Mailing list for Cargo Tracker project  Eclipse Foundation   
8477         Provide convenient MultiStatus constructor     Eclipse Project   

             pd            co   rp          os     